# Vehicle Database Filler

By: Austin Jenkins
Class: Unsupervised Algorithms in Machine Learning

## Table of Contents <a name="tableofcontents"></a>
1. [Table of Contents](#tableofcontents)
2. [Introduction](#Introduction)
3. [Data](#Data)

## Introduction <a name="Introduction"></a>
[Table of Contents](#tableofcontents)

### Background
I have a small company that I run as a project that gathers information about GM vehicles.  Within this company, we parse PDF invoices and extract important information from them.  Since this data comes across as a list of strings, we need to convert this data into a structured format.  I've been doing this in PHP and extracting the data based on the position of the data in the string.  This can sometimes lead to incorrect results.  So, I want to try to use machine learning to extract the data from the strings and fill in any missing parts.

### About the Data

I currently have a MySQL database that contains many tables, but the one we're most interested in is the `vehicles` table.  This table contains the following columns:
- vin - A 17 digit string that is unique to each vehicle
- year - The year the vehicle was made
- make - The make of the vehicle (ex: Chevrolet, GMC, etc.)
- peg_rpo_code - A 3 digit string code that represents the vehicle's trim level
- peg_description - A description of the vehicle's trim level
- exterior_color_rpo_code - A 3 digit string code that represents the vehicle's exterior color
- exterior_color_description - A description of the vehicle's exterior color
- interior_color_rpo_code - A 3 digit string code that represents the vehicle's interior color
- interior_color_description - A description of the vehicle's interior color
- engine_rpo_code - A 3 digit string code that represents the vehicle's engine
- engine_description - A description of the vehicle's engine
- transmission_rpo_code - A 3 digit string code that represents the vehicle's transmission
- transmission_description - A description of the vehicle's transmission

This data does not include the long string from the PDF of the vehicle.  To add this in, I had to write a script within the app which was written in PHP to convert the PDF to a string and append it to each.  This process took quite a while to run.  Within this script I had it output to a csv so that we can use it for our modeling.  This csv is located within the /data/train.csv folder of this repository.

We started collecting this data in 2019 so the data is quite large considering the invoice string is long.  The initial data set is XXX columns and a total size of YYY.

### Problem Description

Given the long string of the invoice, we want to predict the missing values in the `vehicles` table.  We will use the `train.csv` file to train our model and the `test.csv` file to test our model. 

## EDA - Exploratory Data Analysis <a name="EDA"></a>
[Table of Contents](#tableofcontents)

### Load The Data and Libraries

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display, HTML


train_data_frame = pd.read_csv("data/output.csv")

### Initial Data Inspection

In [3]:
print(train_data_frame.info())
print(train_data_frame.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21033 entries, 0 to 21032
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   vin                         21032 non-null  object 
 1   year                        21032 non-null  float64
 2   make                        20753 non-null  object 
 3   peg_rpo_code                21033 non-null  object 
 4   peg_description             21033 non-null  object 
 5   exterior_color_rpo_code     21032 non-null  object 
 6   exterior_color_description  21032 non-null  object 
 7   interior_color_rpo_code     20997 non-null  object 
 8   interior_color_description  20997 non-null  object 
 9   engine_rpo_code             20719 non-null  object 
 10  engine_description          20719 non-null  object 
 11  transmission_rpo_code       21029 non-null  object 
 12  transmission_description    21029 non-null  object 
 13  invoice                     210

As we can see, there's an initial count of 21,033 entries.  However, I suspect there are some duplicates and potential missing values in the vin column.  This is our primary unique identifier so we need to make sure that there are no duplicates.

### Clean The Data
 - Remove Duplicates from the vin (first determine how many duplicates there are)
 - Remove empty invoice rows
 - Remove rows where the invoice column includes the word "available"

In [ ]:
def data_cleanup(data_frame):
    data_frame = data_frame.dropna(subset=['vin', 'peg_rpo_code'])
    data_frame = data_frame.drop_duplicates(subset='vin')
    return data_frame

train_data_frame = data_cleanup(train_data_frame)
print(train_data_frame.info())
print(train_data_frame.head())

<class 'pandas.core.frame.DataFrame'>
Index: 17347 entries, 0 to 21032
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   vin                         17347 non-null  object 
 1   year                        17347 non-null  float64
 2   make                        17129 non-null  object 
 3   peg_rpo_code                17347 non-null  object 
 4   peg_description             17347 non-null  object 
 5   exterior_color_rpo_code     17346 non-null  object 
 6   exterior_color_description  17346 non-null  object 
 7   interior_color_rpo_code     17322 non-null  object 
 8   interior_color_description  17322 non-null  object 
 9   engine_rpo_code             17097 non-null  object 
 10  engine_description          17097 non-null  object 
 11  transmission_rpo_code       17345 non-null  object 
 12  transmission_description    17345 non-null  object 
 13  invoice                     17347 no

### Visualizations of Data

#### Categorical Distributions

Here I'd like to see the distribution of the categorical data.  This will help us understand the data better and see if there are any potential outliers.

In [36]:
def generate_top20_tables(df, count_column):
    """
    Calculates and returns the data I can use to generate the top 20 tables.
    """
    tables = {}
    for make, group in df.groupby('make'):
        counts = group[count_column].value_counts()

        top_20 = counts.head(20).reset_index()
        top_20.columns = [count_column, 'Count']
        top_20['Rank'] = np.arange(1, len(top_20) + 1)
        top_20['Percentage'] = ((top_20['Count'] / counts.sum()) * 100).round(2)

        others_sum = counts.iloc[20:].sum()
        if others_sum > 0:
            others_row = {
                count_column: 'Others',
                'Count': others_sum,
                'Rank': '21+',
                'Percentage': round((others_sum / counts.sum()) * 100, 2)
            } 
            top_20 = pd.concat([top_20, pd.DataFrame([others_row])], ignore_index=True)

        tables[make] = top_20

    return tables


def display_tables_side_by_side(tables):
    """
    Just a helper function to display the tables side by side.
    """
    html = f"<div style='display:flex; justify-content:space-around;'>"
    for make, table in tables.items():
        html += f"<div><h3>{make}</h3>{table.to_html(index=False)}</div>"
    html += "</div>"
    display(HTML(html))

tables = generate_top20_tables(train_data_frame, 'peg_rpo_code')
display_tables_side_by_side(tables)

tables = generate_top20_tables(train_data_frame, 'exterior_color_rpo_code')
display_tables_side_by_side(tables)





peg_rpo_code,Count,Rank,Percentage
1SL,888,1,34.45
1SB,441,2,17.11
1SD,413,3,16.02
1SU,321,4,12.45
1SC,165,5,6.40
1SP,161,6,6.25
1SN,83,7,3.22
G03,43,8,1.67
G02,39,9,1.51
G04,24,10,0.93


exterior_color_rpo_code,Count,Rank,Percentage
GAZ,463,1,17.96
GB8,264,2,10.24
GB0,185,3,7.18
GZB,182,4,7.06
GFM,177,5,6.87
GEJ,158,6,6.13
GP5,121,7,4.69
GZA,112,8,4.34
G1W,105,9,4.07
GXD,92,10,3.57


#### Correlation Analysis